In [1]:
import cPickle as pickle
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [2]:
test = pd.read_csv('./test.csv')

In [3]:
test = pd.DataFrame([['1', 'I\'ll bomb you hisham'],
                     ['2', 'asshole'],
                     ['3', 'whats up?'],
                     ['4', 'nothing much, Im depressed'],
                     ['5','My teacher hates me, never listens to my reasoning'],
                     ['6','I feel like killing her'],
                     ['7','I want to murder her'],
                     ['8','I want to murder her ass'],
                     ['9','idk'],
                     ['10','I\'ll probably murder her']
                    ], columns=['id', 'comment_text'])

In [4]:
test['comment_text'].fillna("unknown", inplace=True)

In [5]:
import re,string
re_tok = re.compile(re.escape(string.punctuation))

In [6]:
def tokenize(s): return re_tok.sub(r' \1 ', s).split()

In [7]:
vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
               min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
               smooth_idf=1, sublinear_tf=1 )
vec = pickle.load(open('/home/ammar/study/8th semester/FYP/toxic comments/final/vec', 'rb'))
test_term_doc = vec.transform(test['comment_text'])

In [8]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)
# x = trn_term_doc
test_x = test_term_doc

In [9]:
label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
preds = np.zeros((len(test), len(label_cols)))

In [11]:
for i, j in enumerate(label_cols):
    print('fit', j)
    m = pickle.load(open('/home/ammar/study/8th semester/FYP/toxic comments/final/model'+ str(i), 'rb'))
    r = pickle.load(open('/home/ammar/study/8th semester/FYP/toxic comments/final/r'+ str(i), 'rb'))
    print r
    preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

('fit', 'toxic')
[[0.57415495 1.24205391 0.9865223  ... 1.79746125 0.75787254 0.75787254]]
('fit', 'severe_toxic')
[[0.99404422 3.59278685 2.95475165 ... 3.66620322 3.10860548 3.10860548]]
('fit', 'obscene')
[[0.55067865 2.09367642 1.53332723 ... 1.9551819  1.39758416 1.39758416]]
('fit', 'threat')
[[1.61549549 4.8033878  4.1653526  ... 4.87680417 4.31920643 4.31920643]]
('fit', 'insult')
[[0.65812021 1.95563576 1.70010415 ... 2.02905213 1.47145439 1.47145439]]
('fit', 'identity_hate')
[[0.55819766 3.72074054 3.08270534 ... 3.79415691 3.23655917 3.23655917]]


In [12]:
submid = pd.DataFrame({'id': test["id"]})
submission = pd.concat([submid, pd.DataFrame(preds, columns = label_cols)], axis=1)

In [13]:
test

,id,comment_text
0,1,I'll bomb you hisham
1,2,asshole
2,3,whats up?
3,4,"nothing much, Im depressed"
4,5,"My teacher hates me, never listens to my reaso..."
5,6,I feel like killing her
6,7,I want to murder her
7,8,I want to murder her ass
8,9,idk
9,10,I'll probably murder her


In [14]:
submission

,id,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,1,0.059580,0.000906,0.012228,0.000622,0.012411,0.001046
1,2,1.000000,0.007853,0.999999,0.000029,0.999978,0.000146
2,3,0.026783,0.000637,0.002520,0.000128,0.002490,0.000622
3,4,0.052771,0.000717,0.023508,0.000632,0.009721,0.000952
4,5,0.189803,0.000848,0.010143,0.000131,0.013627,0.001427
5,6,0.079242,0.000886,0.005503,0.002563,0.004611,0.000944
6,7,0.224149,0.168738,0.187244,0.089265,0.033473,0.001314
7,8,0.987826,0.267454,0.996174,0.066805,0.703133,0.003069
8,9,0.071801,0.000596,0.004070,0.000132,0.008858,0.000461
9,10,0.061142,0.001159,0.007052,0.003288,0.008480,0.000601
